## Data

In [1]:
import torch
import torch.nn as nn
import torchtext; torchtext.disable_torchtext_deprecation_warning()
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

corpus = [
    "ăn quả nhớ kẻ trồng cây",
    "có chí thì nên"    
]
data_size = len(corpus)

# Define the max vocabulary size and sequence length
vocab_size = 15
sequence_length = 7

In [2]:
# Define tokenizer function
tokenizer = get_tokenizer('basic_english')

# Create a function to yield list of tokens
def yield_tokens(examples):
    for text in examples:
        yield tokenizer(text)

# Create vocabulary
vocab = build_vocab_from_iterator(yield_tokens(corpus),
                                  max_tokens=vocab_size,
                                  specials=["<unk>", "<pad>", "<sos_topic1>", "<sos_topic2>", "<eos>"]
                                 )
vocab.set_default_index(vocab["<unk>"])
vocab.get_stoi()

{'ăn': 14,
 'quả': 11,
 'nên': 10,
 'nhớ': 9,
 'trồng': 13,
 '<eos>': 4,
 'kẻ': 8,
 'chí': 5,
 '<unk>': 0,
 'cây': 6,
 '<sos_topic2>': 3,
 'thì': 12,
 'có': 7,
 '<sos_topic1>': 2,
 '<pad>': 1}

In [3]:
data_x = []
data_y = []

corpus[0] = '<sos_topic1> ' + corpus[0] + ' <eos>'
corpus[1] = '<sos_topic2> ' + corpus[1] + ' <eos>'

for vector in corpus:
    vector = vector.split()
    data_x.append(vector[:-1])
    data_y.append(vector[1:])  

print(data_x)
print(data_y)

[['<sos_topic1>', 'ăn', 'quả', 'nhớ', 'kẻ', 'trồng', 'cây'], ['<sos_topic2>', 'có', 'chí', 'thì', 'nên']]
[['ăn', 'quả', 'nhớ', 'kẻ', 'trồng', 'cây', '<eos>'], ['có', 'chí', 'thì', 'nên', '<eos>']]


In [4]:
# Tokenize and numericalize your samples
def vectorize(x, y, vocab, sequence_length):    
    x_ids = [vocab[token] for token in x][:sequence_length]
    x_ids = x_ids + [vocab["<pad>"]] * (sequence_length - len(x))

    y_ids = [vocab[token] for token in y][:sequence_length]
    y_ids = y_ids + [vocab["<pad>"]] * (sequence_length - len(y))
    
    return x_ids, y_ids

# Vectorize the samples
data_x_ids = []
data_y_ids = []
for x, y in zip(data_x, data_y):
    x_ids, y_ids = vectorize(x, y, vocab, sequence_length)
    data_x_ids.append(x_ids)
    data_y_ids.append(y_ids)

In [5]:
for x, y in zip(data_x_ids, data_y_ids):
    print(x)
    print(y)
    print()

[2, 14, 11, 9, 8, 13, 6]
[14, 11, 9, 8, 13, 6, 4]

[3, 7, 5, 12, 10, 1, 1]
[7, 5, 12, 10, 4, 1, 1]



In [6]:
data_x_ids = torch.tensor(data_x_ids, dtype=torch.long)
print(data_x_ids.shape)

torch.Size([2, 7])


In [7]:
data_y_ids = torch.tensor(data_y_ids, dtype=torch.long)
print(data_y_ids.shape)

torch.Size([2, 7])


## Train with full data

In [8]:
class TG_Model(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 
                                      embed_dim)
        self.recurrent = nn.RNN(embed_dim, embed_dim, batch_first=True)
        self.linear = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        x,_ = self.recurrent(x)        
        x = self.linear(x)
        return x.permute(0,2,1)

model = TG_Model(vocab_size, 8)
print(model)

TG_Model(
  (embedding): Embedding(15, 8)
  (recurrent): RNN(8, 8, batch_first=True)
  (linear): Linear(in_features=8, out_features=15, bias=True)
)


In [9]:
outputs = model(data_x_ids)
print(outputs.shape)

torch.Size([2, 15, 7])


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

In [11]:
# train
for _ in range(40):
    optimizer.zero_grad()
    outputs = model(data_x_ids)
    loss = criterion(outputs, data_y_ids)
    print(loss.item())
    loss.backward()
    optimizer.step()

2.7285425662994385
2.327491521835327
2.024104118347168
1.7615522146224976
1.5158345699310303
1.2889503240585327
1.0819638967514038
0.8945137858390808
0.7275064587593079
0.5823119282722473
0.46098971366882324
0.36218732595443726
0.2834357023239136
0.22332413494586945
0.17835228145122528
0.1438109427690506
0.11679001897573471
0.09547559171915054
0.07857147604227066
0.06511998176574707
0.05439886078238487
0.04584752395749092
0.03902139887213707
0.033563174307346344
0.029182540252804756
0.025643642991781235
0.02275848016142845
0.020381955429911613
0.018404319882392883
0.01674303039908409
0.015335754491388798
0.014134482480585575
0.013102053664624691
0.012209041975438595
0.011432068422436714
0.010752348229289055
0.01015467755496502
0.00962649006396532
0.00915764831006527
0.008739611133933067


In [12]:
outputs = model(data_x_ids)
#print(outputs)
print(torch.argmax(outputs, axis=1))

tensor([[14, 11,  9,  8, 13,  6,  4],
        [ 7,  5, 12, 10,  4,  1,  1]])


In [13]:
data_y_ids

tensor([[14, 11,  9,  8, 13,  6,  4],
        [ 7,  5, 12, 10,  4,  1,  1]])

## Inference

In [18]:
promt = '<sos_topic1>'
promt = promt.split()
promt_ids = [vocab[token] for token in promt][:sequence_length]
promt_ids = promt_ids + [vocab["<pad>"]] * (sequence_length - len(promt))

print(promt_ids)

[2, 1, 1, 1, 1, 1, 1]


In [19]:
for i in range(sequence_length - len(promt)):
    promt_tensor = torch.tensor(promt_ids, dtype=torch.long).reshape(1, -1)
    outputs = model(promt_tensor)
    outputs = torch.argmax(outputs, axis=1)   
    next_id = outputs[0][len(promt)+i-1]

    promt_ids[len(promt)+i] = next_id.item()
    print(promt_ids)

[2, 14, 1, 1, 1, 1, 1]
[2, 14, 11, 1, 1, 1, 1]
[2, 14, 11, 9, 1, 1, 1]
[2, 14, 11, 9, 8, 1, 1]
[2, 14, 11, 9, 8, 13, 1]
[2, 14, 11, 9, 8, 13, 6]


In [ ]:
{'ăn': 14,
 'quả': 11,
 'nên': 10,
 'nhớ': 9,
 'trồng': 13,
 '<eos>': 4,
 'kẻ': 8,
 'chí': 5,
 '<unk>': 0,
 'cây': 6,
 '<sos_topic2>': 3,
 'thì': 12,
 'có': 7,
 '<sos_topic1>': 2,
 '<pad>': 1}